In [36]:
%pip install eep153_tools
%pip install python_gnupg
%pip install -U gspread_pandas
#load in file from class
def format_id(id,zeropadding=0):
    """Nice string format for any id, string or numeric.

    Optional zeropadding parameter takes an integer
    formats as {id:0z} where
    """
    if pd.isnull(id) or id in ['','.']: return None

    try:  # If numeric, return as string int
        return ('%d' % id).zfill(zeropadding)
    except TypeError:  # Not numeric
        return id.split('.')[0].strip().zfill(zeropadding)
    except ValueError:
        return None

data_url = "https://docs.google.com/spreadsheets/d/1GTo423_gUJe1Von9jypWAbC0zSQ7WGegAWPuRi7eJAI/edit?gid=1410082681#gid=1410082681"

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [37]:
import pandas as pd
import re 
from eep153_tools.sheets import read_sheets

#create recipes df
recipes = read_sheets(data_url, sheet="recipes")
recipes = (recipes
           .assign(parent_foodcode = lambda df: df["parent_foodcode"].apply(format_id),
                   ingred_code = lambda df: df["ingred_code"].apply(format_id))
           .rename(columns={"parent_desc": "recipe"}))
recipes.head()


,parent_foodcode,recipe,ingred_code,ingred_desc,ingred_wt
0,11340000,"Imitation milk, non-soy, sweetened",43543,"Milk, imitation, non-soy",100.0
1,11460150,"Yogurt, frozen, NS as to flavor, lowfat milk",1298,"Yogurt, frozen, flavors other than chocolate, ...",100.0
2,11460160,"Yogurt, frozen, chocolate, lowfat milk",1117,"Yogurt, plain, low fat, 12 grams protein per 8...",81.8
3,11460160,"Yogurt, frozen, chocolate, lowfat milk",19166,"Cocoa, dry powder, unsweetened, processed with...",5.2
4,11460160,"Yogurt, frozen, chocolate, lowfat milk",19335,"Sugars, granulated",13.0


In [47]:
#List of non-vegan keywords AND non-natural foods keywords (including frozen, processed, etc).
NON_VEGAN_KEYWORDS = [
    "beef", "pork", "chicken", "turkey", "fish", "seafood", "shellfish", "shrimp", "crab","crabs",
    "lamb", "goat", "duck", "goose", "tuna", "salmon", "cod", "bacon", "ham",
    "shellfish", "lobster", "mussels", "oysters", "scallops", "octopus", "eel",
    "organ meat", "milk","Eggnog" "cheese", "butter", "cream","ice cream", "yogurt", "whey",
    "casein", "lactose", "ghee", "buttermilk", "egg", "eggs", "mayo", "mayonnaise", "albumen",
    "albumin", "lysozyme", "ovomucoid", "ovomucin", "ovovitellin", "honey",
    "bee pollen", "royal jelly", "propolis", "shellac", "confectioner’s glaze",
    "carmine", "cochineal", "lard", "tallow", "suet", "gelatin", "collagen",
    "isinglass", "bone broth", "bone stock", "fish sauce", "oyster sauce",
    "shrimp paste", "worcestershire sauce", "anchovies", "rennet", "pepsin",
    "bone char", "vitamin d3", "lanolin", "omega-3 fish oil", "caseinate",
    "lecithin (egg)", "cysteine", "l-cysteine", "glycerin (animal)",
    "glycerol (animal)", "stearic acid (animal)", "tallowate", "sodium tallowate",
    "capric acid", "caprylic acid", "cheese", "pudding", "processed", "veal",'sirloin', "steak", "animal",
    "Custard", "Mousse", "chocolate", "Meatballs", "meat", "Gravy", "poultry","baby", "frozen", 'dairy', 'lump',"peas","school"
]

#this partal match: "milkshake" or "eggroll" will get flagged (since "milk" or "egg" is in the keyword list).
NON_VEGAN_PATTERN = re.compile(
    '|'.join(map(re.escape, NON_VEGAN_KEYWORDS)),
    re.IGNORECASE
)

def filter_vegan_ingredients(df: pd.DataFrame) -> pd.DataFrame:
    # 1) Convert to string, lowercase, remove punctuation
    df["recipe"] = df["recipe"].astype(str).str.lower().fillna("")
    df["recipe"] = df["recipe"].str.replace(r"[^\w\s]", "", regex=True)

    df["ingred_desc"] = df["ingred_desc"].astype(str).str.lower().fillna("")
    df["ingred_desc"] = df["ingred_desc"].str.replace(r"[^\w\s]", "", regex=True)

    # 2) Create a mask for rows that do NOT contain non-vegan keywords
    mask = ~(df["recipe"].str.contains(NON_VEGAN_PATTERN, na=False, regex=True) |
             df["ingred_desc"].str.contains(NON_VEGAN_PATTERN, na=False, regex=True))

    return df[mask]

In [48]:
vegan_recipes = filter_vegan_ingredients(recipes)
vegan_recipes.shape

(11137, 5)

In [49]:
#start copying code from mini lecture VEGAN

#create nutrition df
nutrition = (read_sheets(data_url, sheet="nutrients")
             .assign(ingred_code = lambda df: df["ingred_code"].apply(format_id)))

display(nutrition.head())
nutrition.columns
nutrition.shape



# normalize weights to percentage terms. 
vegan_recipes['ingred_wt'] = vegan_recipes['ingred_wt']/vegan_recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")

# we're going to extend the recipes data frame to include the nutrient profiles of its ingredients (in 100g)
df_vegan = vegan_recipes.merge(nutrition, how="left", on="ingred_code")

# multiply all nutrients per 100g of an ingredient by the weight of that ingredient in a recipe.
numeric_cols = list(df_vegan.select_dtypes(include=["number"]).columns)
numeric_cols.remove("ingred_wt")
df_vegan[numeric_cols] = df_vegan[numeric_cols].mul(df_vegan["ingred_wt"], axis=0)

# sum nutrients of food codes (over the multiple ingredients)
# python tip: one can merge dictionaries dict1 dict2 using **, that is: dict_merge = {**dict1, **dict2}. The ** effectively "unpacks" the key value pairs in each dictionary
df_vegan = df_vegan.groupby('parent_foodcode').agg({**{col: "sum" for col in numeric_cols},
                                        "recipe": "first"})

df_vegan.index.name = "recipe_id"

food_names = df_vegan["recipe"]
print(food_names.head())
df_vegan.head()
df_vegan.shape

,ingred_code,Ingredient description,Capric acid,Lauric acid,Myristic acid,Palmitic acid,Palmitoleic acid,Stearic acid,Oleic acid,Linoleic Acid,...,Vitamin B12,"Vitamin B-12, added",Vitamin B6,Vitamin C,Vitamin D,Vitamin E,"Vitamin E, added",Vitamin K,Water,Zinc
0,1001,"Butter, salted",2.529,2.587,7.436,21.697,0.961,9.999,19.961,2.728,...,0.17,0.0,0.003,0.0,0.0,2.32,0.0,7.0,15.87,0.09
1,1002,"Butter, whipped, with salt",2.039,2.354,7.515,20.531,1.417,7.649,17.370,2.713,...,0.07,0.0,0.008,0.0,0.0,1.37,0.0,4.6,16.72,0.05
2,1003,"Butter oil, anhydrous",2.495,2.793,10.005,26.166,2.228,12.056,25.026,2.247,...,0.01,0.0,0.001,0.0,0.0,2.80,0.0,8.6,0.24,0.01
3,1004,"Cheese, blue",0.601,0.491,3.301,9.153,0.816,3.235,6.622,0.536,...,1.22,0.0,0.166,0.0,0.5,0.25,0.0,2.4,42.41,2.66
4,1005,"Cheese, brick",0.585,0.482,3.227,8.655,0.817,3.455,7.401,0.491,...,1.26,0.0,0.065,0.0,0.5,0.26,0.0,2.5,41.11,2.60


recipe_id
11115400    kefir ns as to fat content
11440060                  tzatziki dip
11551050             licuado or batido
11553100            fruit smoothie nfs
11710000            infant formula nfs
Name: recipe, dtype: object


/tmp/ipykernel_160/3073712158.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vegan_recipes['ingred_wt'] = vegan_recipes['ingred_wt']/vegan_recipes.groupby(['parent_foodcode'])['ingred_wt'].transform("sum")


(3893, 66)

In [50]:
prices = read_sheets(data_url, sheet="prices")[["food_code", "year", "price"]]

prices["food_code"] = prices["food_code"].apply(format_id)

prices = prices.set_index(["year", "food_code"])
print(prices.index.levels[0])

# we'll focus on the latest price data
prices = prices.xs("2017/2018", level="year")

# drop rows of prices where the price is "NA"
prices = prices.dropna(subset="price")

print(f"We have prices for {prices.shape[0]} unique recipes (FNDDS food codes)")

Index(['2011/2012', '2013/2014', '2015/2016', '2017/2018'], dtype='object', name='year')
We have prices for 4435 unique recipes (FNDDS food codes)


In [51]:
#add diet requirements

rda = read_sheets(data_url, sheet="rda")

rda = rda.set_index("Nutrient")
rda_min = rda[rda["Constraint Type"].isin(["RDA", "AI"])].copy()


rda.columns, rda.head(), rda_min.head()

(Index(['Nutrient Type', 'Unit', 'Constraint Type', 'Female_19_30',
        'Female_endurance_athlete', 'Male_19_30', 'Male_endurance_athlete'],
       dtype='object'),
               Nutrient Type  Unit Constraint Type  Female_19_30  \
 Nutrient                                                          
 Energy                Macro  kcal             RDA        2000.0   
 Protein               Macro     g             RDA          46.0   
 Carbohydrate          Macro     g             RDA         130.0   
 Dietary Fiber         Macro     g             RDA          28.0   
 Linoleic Acid         Macro     g              AI          12.0   
 
                Female_endurance_athlete  Male_19_30  Male_endurance_athlete  
 Nutrient                                                                     
 Energy                           2800.0      2400.0                  3500.0  
 Protein                            80.0        56.0                   112.0  
 Carbohydrate                      48

In [52]:
common_recipes = df_vegan.index.intersection(prices.index)

# python tip: given a list of indices, "loc" both subsets and sorts. 
df_vegan = df_vegan.loc[common_recipes]
prices = prices.loc[common_recipes]

# lets remap the price dataframe index to be the actual food names.
prices.index = prices.index.map(food_names)

A_all = df_vegan.T

print(prices.head())
print(A_all.head())

                                                       price
kefir ns as to fat content                          0.345625
tzatziki dip                                        1.217789
licuado or batido                                   0.189099
fruit smoothie nfs                                  0.462558
infant formula readytofeed similac expert care ...  1.074909
                 11115400  11440060  11551050  11553100 11710051 11710055  \
Capric acid        0.0195  0.005673   0.00048  0.000813    0.689    0.689   
Lauric acid         0.026  0.000273  0.000959  0.001626    0.023    0.023   
Myristic acid      0.0945  0.000545  0.000959  0.001626    0.007    0.007   
Palmitic acid      0.2805    5.7604  0.053557  0.083665    0.137    0.137   
Palmitoleic acid   0.0185  0.638909  0.005183  0.008129    0.003    0.003   

                  11710357  11710358 11710377 11710378  ... 95312410 95312560  \
Capric acid       0.078597  0.078597      0.0      0.0  ...      0.0      0.0   
Lauric ac

In [53]:
# pick a demographic (column from rda dataframe)
'''
select from 
['Female_19_30', 'Female_endurance_athlete', 'Male_19_30', 'Male_endurance_athlete']
'''
group = "Female_endurance_athlete"

# create lower bounds and upper bounds.
bmin = rda.loc[rda['Constraint Type'].isin(['RDA', 'AI']), group]
bmax = rda.loc[rda['Constraint Type'].isin(['UL']), group]

# reindex ensures we only keep nutrients in bmin/bmax
Amin = A_all.reindex(bmin.index).dropna(how='all')
Amax = A_all.reindex(bmax.index).dropna(how='all')

b = pd.concat([bmin, -bmax])
A = pd.concat([Amin, -Amax])

#python tip: by typing "=" after the variable name inside the curly braces, it formats the output so we don't have to write f"variable = {variable}"
print(f"{bmin.shape=}")
print(f"{Amin.shape=}")
print(f"{bmax.shape=}")
print(f"{Amax.shape=}")
print(f"{b.shape=}")
print(f"{A.shape=}")
print(f"{prices.shape=}")

bmin.shape=(26,)
Amin.shape=(26, 1834)
bmax.shape=(1,)
Amax.shape=(1, 1834)
b.shape=(27,)
A.shape=(27, 1834)
prices.shape=(1834, 1)


In [54]:
from  scipy.optimize import linprog as lp
import numpy as np
p = prices
tol = 1e-6 # Numbers in solution smaller than this (in absolute value) treated as zeros
result = lp(p, -A, -b, method='highs')


In [55]:
print(f"Cost of diet for a vegan {group} is ${result.fun:.2f} per day.")

Cost of diet for a vegan Female_endurance_athlete is $3.97 per day.


In [56]:
# lets mess with the index on price df so they are recipe names not ids.

# get the result x in a series with food names
diet = pd.Series(result.x,index=prices.index)


print(f"Cost of diet for a vegan {group} is ${result.fun:.2f} per day. \n")

print(f"As a vegan {group} you'll be eating (in 100s of grams or milliliters): \n")

print(round(diet[diet >= tol], 2))

Cost of diet for a vegan Female_endurance_athlete is $3.97 per day. 

As a vegan Female_endurance_athlete you'll be eating (in 100s of grams or milliliters): 

vermicelli made from soybeans                    3.18
peanuts unroasted                                1.04
flax seeds                                       0.02
cereal toasted oat                               3.40
beans and rice with tomatoes                     2.40
ripe plantain raw                                1.81
cilantro raw                                     0.20
nutritional powder mix high protein herbalife    0.07
dtype: float64


In [57]:
#Function for deliverable [A] Dietary Reference Intakes

#NEED TO ADD MORE categories into the Google sheet to make better, females, males of different ages, etc.
def get_population_dri(population, rda_df) -> pd.Series:
    
    # 1. Filter rows to only those where Constraint Type is RDA or AI
    rda_filtered = rda_df[rda_df["Constraint Type"].isin(["RDA", "AI"])].copy()

    # 3. Extract the column for the chosen population as a Series
    dri_series = rda_filtered[population]

    # 4. Drop any rows that are NaN (just in case)
    dri_series.dropna(inplace=True)

    # 5. Return the final Series
    return dri_series


In [58]:
#example of get_population_dri function

'''
select population from 
['Female_19_30', 'Female_endurance_athlete', 'Male_19_30', 'Male_endurance_athlete']
'''

population = "Female_endurance_athlete"
dri_for_female_19_30 = get_population_dri(population, rda)

print("Dietary recommendations (RDA) for", population)
print(dri_for_female_19_30)

Dietary recommendations (RDA) for Female_endurance_athlete
Nutrient
Energy            2800.0
Protein             80.0
Carbohydrate       488.0
Dietary Fiber       28.0
Linoleic Acid       12.0
Linolenic Acid       1.1
Calcium           1000.0
Iron                22.0
Magnesium          310.0
Phosphorus         700.0
Potassium         4700.0
Zinc                 8.0
Copper               0.9
Selenium            55.0
Vitamin A          700.0
Vitamin E           15.0
Vitamin D           25.0
Vitamin C           75.0
Thiamin              1.1
Riboflavin           1.1
Niacin              14.0
Vitamin B6           1.3
Vitamin B12          2.4
Choline            425.0
Vitamin K           90.0
Folate             400.0
Name: Female_endurance_athlete, dtype: float64


In [59]:
#test of function to optimize all of code
from scipy.optimize import linprog
def min_cost_diet_with_bmin_bmax_and_prints(
    group: str,
    rda: pd.DataFrame,
    df_vegan: pd.DataFrame,
    prices: pd.DataFrame
) -> linprog:

    # 1) Create lower (bmin) and upper (bmax) bounds for the chosen group
    #    using rows in rda where Constraint Type is RDA or AI.
    bmin = rda.loc[rda["Constraint Type"].isin(["RDA", "AI"]), group]
    bmax = rda.loc[rda["Constraint Type"].isin(["RDA", "AI"]), group]

    # Reindex so they have the same nutrients
    bmin = bmin.reindex(bmax.index).dropna(how="all")
    bmax = bmax.reindex(bmin.index).dropna(how="all")

    # 2) Intersect df_vegan & prices to ensure matching foods
    common_recipes = df_vegan.index.intersection(prices.index)
    df_vegan = df_vegan.loc[common_recipes]
    prices = prices.loc[common_recipes]

    # 3) Build the ">= bmin" constraints
    nutrients = bmin.index.tolist()
    for nut in nutrients:
        if nut not in df_vegan.columns:
            raise ValueError(f"Nutrient '{nut}' not found in df_vegan columns.")

    # Each row in df_vegan is a food; columns are nutrients => shape: (n_foods, n_nutrients)
    A_bmin = df_vegan[nutrients].to_numpy()

    # Convert >= to <= by multiplying by -1
    A_ub = -A_bmin.T  # shape: (n_nutrients, n_foods)
    b_ub = -bmin.values  # shape: (n_nutrients,)

    # 4) Build the cost vector
    if "price" not in prices.columns:
        raise ValueError("The 'prices' DataFrame must have a 'price' column.")
    cost_vector = prices["price"].to_numpy()  # shape: (n_foods,)

    # Each food's serving >= 0
    bounds = [(0, None)] * len(df_vegan)

    # 5) Solve the linear program
    tol = 1e-6  # treat solution values smaller than this as zero
    result = linprog(
        c=cost_vector,
        A_ub=A_ub,
        b_ub=b_ub,
        bounds=bounds,
        method="highs",
        options={"tol": tol}
    )

    if not result.success:
        raise RuntimeError(f"Linear program failed: {result.message}")

    # 6) Print results in your screenshot style
    print(f"Cost of diet for a vegan {group} is ${result.fun:.2f} per day.\n")

    # Build a Series for the solution
    diet = pd.Series(result.x, index=prices.index, name="servings")

    # Print "100s of grams/mL" if 1 serving ~ 100 g/mL
    diet_100 = (diet * 100).round(2)
    print(f"\nAs a vegan {group}, you'll be eating (in 100s of grams or milliliters):\n")
    print(diet_100[diet_100 > 0.01])

    # Return the entire solver result for further inspection
    return result


NameError: name 'linprog' is not defined

In [44]:
population = "Female_endurance_athlete"
servings_series = min_cost_diet_with_remapped_names(
    population=population,
    rda=rda,
    df_vegan=df_vegan,
    prices=prices,
    name_column="recipe"   
)

NameError: name 'min_cost_diet_with_remapped_names' is not defined

In [ ]:
# Package Installations
import pandas as pd
import os
!pip install xlrd

In [ ]:
file_path = '/home/jovyan/Project2/CPI/food_affordability_CA.xls'
affordability_CA = pd.read_excel(file_path)
affordability_CA.head()

In [ ]:
## Regional CPI ##

The followings are the census divisions: 
New England, Middle Atlantic, East North Central, West North Central, South Atlantic, East South Central, West South Central, Mountain, and  Pacific. Each dataframe has the CPI for each census region with the monthly CPI changes as well as annual and first half and second half. 

In [ ]:
#All the file path
file_path_1 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_EastSouthCentral.csv'
file_path_2 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_MiddleAtlantic.csv'
file_path_3 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_East_North_Central.csv'
file_path_4 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_Mountain.csv'
file_path_5 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_NewEngland.csv'
file_path_6 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_Pacific.csv'
file_path_7 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_SouthAtlantic.csv'
file_path_8 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_WestNorthCentral.csv'
file_path_9 = '/home/jovyan/Project2/CPI/Census_Divisions/CPI_WestSouthCentral.csv'

#individual files for each census regions
CPI_East_South_Central = pd.read_csv(file_path_1)
CPI_Middle_Atlantic = pd.read_csv(file_path_2)
CPI_East_North_Central = pd.read_csv(file_path_3)
CPI_Mountain = pd.read_csv(file_path_4)
CPI_NewEngland = pd.read_csv(file_path_5)
CPI_Pacific = pd.read_csv(file_path_6)
CPI_South_Atlantic = pd.read_csv(file_path_7)
CPI_West_North_Central = pd.read_csv(file_path_8)
CPI_West_South_Central = pd.read_csv(file_path_9)

#example of a dataframe
CPI_East_South_Central

In [ ]:
## Clean the Dataframe to make it easier to work with ##

In [ ]:
import pandas as pd

# Dictionary of DataFrames
cpi_dfs = {
    "CPI_East_South_Central": CPI_East_South_Central,
    "CPI_Middle_Atlantic": CPI_Middle_Atlantic,
    "CPI_East_North_Central": CPI_East_North_Central,
    "CPI_Mountain": CPI_Mountain,
    "CPI_NewEngland": CPI_NewEngland,
    "CPI_Pacific": CPI_Pacific,
    "CPI_South_Atlantic": CPI_South_Atlantic,
    "CPI_West_North_Central": CPI_West_North_Central,
    "CPI_West_South_Central": CPI_West_South_Central
}

# Loop through all DataFrames and set "Year" as the index
for name, df in cpi_dfs.items():
    if "Year" in df.columns:  # Check if "Year" column exists
        df.set_index("Year", inplace=True)

# Now all DataFrames have "Year" as the index

CPI_East_North_Central

In [ ]:
2017 is the base year for CPI calculation for all the dataframe. We will use the formula:

In [ ]:
## Calculating Adjusted Cost of Minimum Diet Using CPI

To determine the cost of a minimum diet for each region, we use the Consumer Price Index (CPI) as an adjustment factor. The formula is given by:

$$
\text{Adjusted Cost}_{\text{region}} = \text{Base Cost} \times \frac{\text{CPI}_{\text{region}}}{\text{CPI}_{\text{base}}}
$$

where:
- **$\text{Adjusted Cost}_{\text{region}}$** is the estimated cost of the minimum diet in the specific region.
- **$\text{Base Cost}$** is the reference cost of the minimum diet (typically from a standard region or dataset).
- **$\text{CPI}_{\text{region}}$** is the Consumer Price Index for the specific region.
- **$\text{CPI}_{\text{base}}$** is the CPI of the reference region.

This method adjusts the base cost according to regional price variations, ensuring that the diet cost reflects local economic conditions.


In [ ]:
## Change the variable name and structure AFTER sorting out the FINAL files ##

In [ ]:
min_price_vegan_m_endurance_ath = result.fun
diet_vegan_m_endurance_ath = diet[diet >= tol]
min_price_vegan_m_endurance_ath
#diet_vegan_m_endurance_ath

In [ ]:
## The Calculated Adjusted Minimum Cost Diet as a Dataframe ##

In [ ]:
CPI_East_North_Central.head()

In [ ]:
import pandas as pd

# Dictionary containing all CPI DataFrames
cpi_dfs = {
    "East_South_Central": CPI_East_South_Central,
    "Middle_Atlantic": CPI_Middle_Atlantic,
    "East_North_Central": CPI_East_North_Central,
    "Mountain": CPI_Mountain,
    "NewEngland": CPI_NewEngland,
    "Pacific": CPI_Pacific,
    "South_Atlantic": CPI_South_Atlantic,
    "West_North_Central": CPI_West_North_Central,
    "West_South_Central": CPI_West_South_Central
}

# Define base year and minimum cost of diet
base_year = 2017  #  CPI base year for all census region
min_price_vegan_m_endurance_ath = min_price_vegan_m_endurance_ath 

# Create an empty list to store results
adjusted_cost_data = []

# Loop through each region's CPI data
for region, df in cpi_dfs.items():
    # Ensure Year is the index and fetch CPI values
    if base_year in df.index and 2024 in df.index:
        cpi_base = df.loc[base_year, "Dec"]  # CPI for base year is given in December
        cpi_latest = df.loc[2024, "Annual"]  # CPI for latest available year which is 2024
        # Calculate adjusted diet cost
        adjusted_cost = min_price_vegan_m_endurance_ath * (cpi_latest / cpi_base)
        
        # Append results
        adjusted_cost_data.append([region, adjusted_cost])

# Convert results to DataFrame
adjusted_cost_df = pd.DataFrame(adjusted_cost_data, columns=["Region", "Adjusted_Cost"])

# Display the DataFrame in Jupyter Notebook
display(adjusted_cost_df)  # For Jupyter Notebook

In [ ]:
def adjust_cost_by_cpi(min_price, cpi_dfs, base_year=2017, latest_year=2024):
    """
    Adjusts a base diet cost using CPI data from multiple regions and returns
    a DataFrame with the region name and adjusted costs.

    Parameters:
    -----------
    min_price : float
        Minimum diet cost to be adjusted (e.g., min_price_vegan_m_endurance_ath)
    cpi_dfs : dict of pd.DataFrame
        Dictionary keyed by region. Each value is a DataFrame with CPI data for multiple years,
        where the DataFrame index is years, and includes columns like 'Dec' and 'Annual'
    base_year : int, default 2017
        The year in the DataFrame(s) to use as a CPI base.
    latest_year : int, default 2024
        The year in the DataFrame(s) to use as the CPI for the latest adjustment.

    Returns:
    --------
    pd.DataFrame
        A DataFrame with columns ["Region", "Adjusted_Cost"] where "Region" is the key
        from the dictionary, and "Adjusted_Cost" is the adjusted cost for that region.
    """
    adjusted_cost_data = []

    for region, df in cpi_dfs.items():
        # We check both the base and latest years are present in the index
        if base_year in df.index and latest_year in df.index:
            cpi_base = df.loc[base_year, "Dec"]
            cpi_latest = df.loc[latest_year, "Annual"]
            
            # Calculate the adjusted cost
            adjusted_cost = min_price * (cpi_latest / cpi_base)
            
            # Append to our list of results
            adjusted_cost_data.append([region, adjusted_cost])

    # Convert to DataFrame
    adjusted_cost_df = pd.DataFrame(adjusted_cost_data, columns=["Region", "Adjusted_Cost"])

    return adjusted_cost_df
    